In [1]:
pip install openai pydantic -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
import json

from pydantic import BaseModel, Field
from typing import Optional, List
from openai import OpenAI
import pandas as pd

pd.set_option('display.max_colwidth', 300)

Pydantic is a library that helps  define the shape and type of our data, making sure it matches our expectations before we proceed with our code. Here's a breakdown of the code block using Pydantic:

- **BaseModel**: This is a key part of Pydantic. Think of it as a blueprint for the data you expect. You define a class that inherits from `BaseModel`, and in it, you detail the structure of your data.

- **Fields**: Inside each class that inherits from `BaseModel`, we define what data we expect using fields. For example, `content: str` means we expect a piece of data named `content` that should be a string. The `Field(...)` function is used to add extra details about each field, like a description or default value.

- **Classes Defined**:
  - **SummaryTweet**: A class for creating objects that represent a concise version of an abstract.
  - **AnnounceTweet**: A class for making objects that represent a tweet announcing a new article.
  - **InsightTweet**: This class is for objects that encapsulate a simple insight from complex social science concepts.
  - **ThreadTweet**: A class for a list of tweets, making up a thread about an article.
  - **GenerateTweets**: This class combines all the above, organizing them into lists of each type of tweet.

- **List Fields**: When we want to store multiple items (like multiple tweets of the same kind), we use `List[SomeClass]`. This tells Pydantic we expect a list where each item is an instance of `SomeClass`.

This code essentially sets up a structure for handling different types of tweets as data objects. By defining these classes, we ensure that the data we work with matches our expectations, reducing bugs and making our code more reliable.


In [42]:
class SummaryTweet(BaseModel):
    content: str = Field(..., description="A concise version of the abstract in tweet format. No Hastags")

class AnnounceTweet(BaseModel):
    content: str = Field(..., description="A tweet announcing a new article. Text must include the author and title.  No Hastags")

class InsightTweet(BaseModel):
    content: str = Field(..., description="Simple, understandable statement distilling a complex social science concept.  No Hastags")

class ThreadTweet(BaseModel):
    content: List[str] = Field(default=[], description="A list of tweets that together form a thread about the article, summarizing its key points and implications, engaging the audience, and encouraging further discussion or reading. No hashtags.")

class GenerateTweets(BaseModel):
    summary_tweets: List[SummaryTweet] = Field(default=[], description="A list of SummaryTweet instances.")
    insight_tweets: List[InsightTweet] = Field(default=[], description="A list of InsightTweet instances.")
    announce_tweets: List[AnnounceTweet] = Field(default=[], description="A list of AnnounceTweet instances.")
    thread_tweets: List[ThreadTweet] = Field(default=[], description="A list of ThreadTweet instances.")

# Note: there might be a simpler way to do this, since each of the subclasses only has one field.

For our purposes, we are using to create a JSON schema that we want ChatGPT to follow.

In [70]:
ThreadTweet.model_json_schema()

{'properties': {'content': {'default': [],
   'description': 'A list of tweets that together form a thread about the article, summarizing its key points and implications, engaging the audience, and encouraging further discussion or reading. No hashtags.',
   'items': {'type': 'string'},
   'title': 'Content',
   'type': 'array'}},
 'title': 'ThreadTweet',
 'type': 'object'}

In [71]:
GenerateTweets.model_json_schema()

{'$defs': {'AnnounceTweet': {'properties': {'content': {'description': 'A tweet announcing a new article. Text must include the author and title.  No Hastags',
     'title': 'Content',
     'type': 'string'}},
   'required': ['content'],
   'title': 'AnnounceTweet',
   'type': 'object'},
  'InsightTweet': {'properties': {'content': {'description': 'Simple, understandable statement distilling a complex social science concept.  No Hastags',
     'title': 'Content',
     'type': 'string'}},
   'required': ['content'],
   'title': 'InsightTweet',
   'type': 'object'},
  'SummaryTweet': {'properties': {'content': {'description': 'A concise version of the abstract in tweet format. No Hastags',
     'title': 'Content',
     'type': 'string'}},
   'required': ['content'],
   'title': 'SummaryTweet',
   'type': 'object'},
  'ThreadTweet': {'properties': {'content': {'default': [],
     'description': 'A list of tweets that together form a thread about the article, summarizing its key points and

In [43]:
# I put in the introduction when using GPT4 because it actually uses the extra information.

article = '''
TITLE: The MAGA Movement's Big Umbrella
AUTHOR: Hank Johnston
JOURNAL: Mobilization: An International Quarterly (2024) 28 (4): 409–433.
LINK: https://doi.org/10.17813/1086-671X-28-4-409

ABSTRACT: This article considers the phenomenon of MAGAism as a general, “big-umbrella” social movement to probe its structure and persistence. Drawing on my research on nationalist movements, I discuss the narrative flexibility and emotional power of nationalism and consider how these characteristics fuel a particularly resentful form of majoritarian nationalism—MAGAism. I identify five points of entry for MAGA participation, starting with this bitter majoritarian nationalism, then populism, then traditional conservatism, next a Trumpian personality cult, and ending with the alt-right’s extreme white supremacism and fascism. I then discuss two forces that gather this unlikely collection of groups and individuals under the MAGA umbrella: (1) the flexibility of social identity and how it allows status-threat narratives to subsume and redirect economic and political claims; (2) the social media environment of the 2020s, cut loose from the traditional gatekeepers of news and information, wherein maximizing hits—not accuracy—is the guiding principle. On the one hand, social media intensify the prominence of the demagogic celebrity at the helm of the Republican Party, Donald J. Trump. On the other hand, alt-right trolls compete for prominence by seeking outrageousness and shock value, enhanced by algorithms that create a closed information environment. These social media trends pull MAGA adherents further to the right by inflaming public discourse and building the movement on lies and conspiracies.
INTRODUCTION: The rise of far right-wing parties and extremist groups is now a topic of deep concern among social scientists who study collective action and contentious politics.  As a reader of this special issue of Mobilization, this trend no doubt has been on your mind a lot, as it has been on mine—especially the challenges it poses to democratic institutions.i In the U.S. today, one of the major political parties actively restricts voting rights, bans books in libraries, embraces conspiracies, flaunts constitutional principles, and limits public-school curricula on topics such as diversity, slavery, gender and sexuality (Natanson 2023a, 2023b)—and millions of MAGA supporters are okay with it (Baker, Perry, and Whitehead, 2020; Enders, Farhart, Miller, and Uscinski 2022; Graham et al. 2021; Stewart 2023). Further along the right-wing spectrum, one increasingly finds rising antisemitism, white Christian nationalism, gun-toting bullies, and misogynists (see Davis and Kettrey’s study in this issue) in the U.S. public sphere, raising parallels with European history in the early twentieth century and its brown shirts and black shirts (rather than today’s polo shirts and tac gear). While violence and intolerance are not unique to the extreme right, these trends reflect a synergy between the current political environment, eroding democratic norms, and—among more and more citizens in democratic states—authoritarian and even fascist tendencies (Jackson 2021; Wintemute, Robinson, Tomish 2022). In the U.S. these trends can be grouped under the umbrella of MAGAism where majoritarian nationalism mixes with populism, far-right extremism, and a virtual (in both meanings of the term) potpourri of themes such as antiwokeness, antivaccines, parental rights, gun rights, sovereign citizenry, science denial, masculinity, the lie of a stolen 2020 election, Trumpian personality cult, white replacement, QAnon conspiracies, among many others. This essay contextualizes the special issue articles by situating them in relation to trends in the U.S., which I discuss under the big umbrella of MAGAism. Following Tarrow (2021: 175), common usage in the popular press, and Trump himself, I consider MAGAism as a general social movement, one that encompasses the rightward shift of public discourse and politics in the U.S. plus the array of mobilizing groups that drive it.ii  I will offer a somewhat personalized essay that draws on insights from my work on the power (and danger) of nationalism to analyze a particularly resentful majoritarian nationalism—often labeled white nationalism in the U.S.  (Graham et al. 2021; Mudde 2019, Miller-Idriss, 2021; Young 2017). I am interested not only in militias, alt-right trolls, and white-supremacist churches,  but—more broadly—the mechanisms that allow these extremists to be grouped with millions of middle-class working citizens who come to the movement via parental-rights groups, suburban bible-study groups, or as antivaxxers, tradition-alists, or Constitutionalists.  MAGAism is a big-tent collective action—a movement with continuity, shared identity, diverse claims and political objectives, networked by an array of groups and organizations—the movement’s mobilizing structures that mostly stand apart from the Republican Party. I will identify the movement’s patterns of discourse and how it is structured to help understand the complexity, die-hard loyalty, denialism, and conspiracies that unite millions of MAGAistas in the face of factual evidence that should undermine their beliefs. I pose the fundamental question of collective action: how is the MAGA movement possible?  
'''

In [72]:
def make_tweets(article):
    client = OpenAI(
        max_retries=3,
        timeout=90.0, 
    )

    messages = [
        {
            "role": "system",
            "content": ''' You write a variety of different types of social media posts about sociology articles.
If you perform well, everyone gets a raise, while there are negative consequences if the posts don't get views.
NEVER USE HASHTAGS.
''',
        },
        {
            "role": "user",
            "content": f"""Generate five tweets of each style plus a tweet thread about this sociology article:

            
      {article}
      
      """,
        },
    ]

    completion = client.chat.completions.create(
        model = 'gpt-4-turbo-preview',     #   model="gpt-3.5-turbo" is 20x cheaper but isn't as insightful 
        functions=[
            {
                "name": "generate_tweets",
                "description": "Create multiple social media posts, including a tweet thread, based on an article abstract.",
                "parameters": GenerateTweets.model_json_schema(),
            },
        ],
        n=1,
        messages=messages,
    )
    r = json.loads(completion.choices[0].message.function_call.arguments)
    tweets = [{'type': tweet_type.split('_')[0], 'text': t['content']} for tweet_type in r for t in r[tweet_type]]

    return pd.DataFrame(tweets)


In [66]:
tdf = make_tweets(article)

In [67]:
# Grouping and printing with check for list in 'text' column
for type_name, group in tdf.groupby('type'):
    print(f"Type: {type_name}")
    for index, row in group.iterrows():
        if isinstance(row['text'], list):
            for item in row['text']:
                print(f"- {item}")
        else:
            print(f"- {row['text']}")


Type: announce
- New research by Hank Johnston in Mobilization investigates the MAGA movement's vast reach and the social forces fueling its endurance. The MAGA Movement's Big Umbrella: https://doi.org/10.17813/1086-671X-28-4-409
Type: insight
- Nationalism's emotional power can forge unlikely alliances, proving pivotal in the rise of movements like MAGAism.
- Social identity's flexibility allows economic and political grievances to be reframed as status threats, fueling movements like MAGA.
- In an era where hits trump accuracy, social media perpetuates echo chambers that magnify polarizing figures and ideologies.
- The MAGA movement’s diverse entry points reflect the complex appeal of political identities in today's rapidly changing society.
- Extremist movements leverage the digital age's vast reach, showcasing the dramatic impact of social media on political mobilization.
Type: summary
- Hank Johnston's article in Mobilization delves into MAGAism, exploring its roots in resentful m

GPT4 doesn't always follow the directions, for example, there's only one "announce" style tweet, but GPT3.5 gets lost more often when something complex is going on, such as the tweet thread.